In [1]:
import math
import os
import re
import nltk
import numpy as np
import pandas as pd
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path

In [2]:
tweets = pd.read_csv('input/data.csv')

In [3]:
tweets.head()

,id,text,place,retweet_count,favorite_count,num_hashtags,num_urls,num_mentions,label
0,1,RT @morningJewshow: Speaking about Jews and co...,RRN,1,0,0,0,1,ham
1,2,This age/face recognition thing..no reason pla...,SILK,0,0,0,0,0,ham
2,3,Only upside of the moment I can think of is th...,BTM,0,0,2,0,0,ham
3,4,If you're going to think about+create experien...,SILK,2,1,2,0,0,ham
4,5,Watching a thread on FB about possible future ...,KOR,0,0,0,0,0,ham


In [4]:
tweets = tweets[['text','place','label']]

In [5]:
tweets.groupby('label').describe()

text                                                                 \
      count unique                                                top freq   
label                                                                        
ham    1000    988                             Night good people. #fb   11   
spam    519    519  RT @aaron_glaser: 2/13 SLOW DANCE COMEDY NIGHT...    1   

      place                   
      count unique  top freq  
label                         
ham    1000      9  RRN  123  
spam    519      9  JPN   65

In [6]:
cleanup_re = re.compile('[^a-z]+')

In [7]:
def cleanup(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = cleanup_re.sub(' ', sentence).strip()
    return sentence

In [8]:
tweets["text"] = tweets["text"].apply(cleanup)

In [9]:
tweets.to_csv('processed/tweets_all.csv', columns=['text','place','label'], index_label= 't_id')

# Split Ham and Spam

In [10]:
tweets_ham = tweets[tweets['label'] == 'ham']['text']

In [11]:
tweets_ham.head(20)

0     rt morningjewshow speaking about jews and come...
1     this age face recognition thing no reason plat...
2     only upside of the moment i can think of is th...
3     if you re going to think about create experien...
4     watching a thread on fb about possible future ...
7     rt thepatrickwalsh kriss kross once rapped som...
9     thelancearthur hotdogsladies i didn t realize ...
11    wkamaubell i stand by you i don t want men tel...
12                           frogtosser thank you becca
20                                      deray thank you
21    rt iamsakuma hundreds of people are in union s...
25    leadership tech when will oths stop with the s...
26    infuriating to have the oakland pd tell me the...
27    the best part of heading to toronto for a mont...
28    oakland tech is once again got the stadium lev...
30    twibnation is grassroots media covering on the...
33    rt camerontw fingers crossed we need good news...
34    how did your grandparents meet mine involv

In [12]:
tweets_ham = pd.DataFrame(tweets_ham)
tweets_ham.to_csv('processed/tweets_ham.csv', columns=['text'], index_label= 't_id')

In [13]:
tweets_spam = tweets[tweets['label'] == 'spam']['text']

In [14]:
tweets_spam = pd.DataFrame(tweets_spam)
tweets_spam.to_csv('processed/tweets_spam.csv', columns=['text'], index_label= 't_id')

In [15]:
tweets_ham.head()

,text
0,rt morningjewshow speaking about jews and come...
1,this age face recognition thing no reason plat...
2,only upside of the moment i can think of is th...
3,if you re going to think about create experien...
4,watching a thread on fb about possible future ...


# Read Files

In [16]:
tweets_ham = pd.read_csv('processed/tweets_ham.csv')

In [17]:
tweets_ham.head()

,t_id,text
0,0,rt morningjewshow speaking about jews and come...
1,1,this age face recognition thing no reason plat...
2,2,only upside of the moment i can think of is th...
3,3,if you re going to think about create experien...
4,4,watching a thread on fb about possible future ...


In [18]:
tweets_spam = pd.read_csv('processed/tweets_spam.csv')

In [19]:
tweets_spam.head()

,t_id,text
0,5,don t ok https t co utxrj zvdn
1,6,rt sanfranciscoba enoughsenough surprise woody...
2,8,rt theatlanticpol watch baltimore native taneh...
3,10,rt sippey here s my one paragraph review of th...
4,13,mmflint i dont ask for rts gen but my friend f...


## Natural Language Processing

In [20]:
import string
from nltk.corpus import stopwords

In [21]:
print(stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\preethi.gupta/nltk_data'
    - 'C:\\Users\\preethi.gupta\\Anaconda3\\nltk_data'
    - 'C:\\Users\\preethi.gupta\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\preethi.gupta\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\preethi.gupta\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
tweets_spam['text'].head()

In [ ]:
tweets_ham['text'].head()

In [ ]:
t_id = list(tweets_spam['t_id'])
type(t_id)

In [ ]:
t_id_ham = list(tweets_ham['t_id'])
type(t_id)

In [ ]:
def clean_text(mess):
   # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    l = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    val = ' '.join(l)
    # Now just remove any stopwords
    return val

In [ ]:
# To hold the word vector of every tweets
Sword_list = []

for i in range(0, len(t_id)):
    tw_msg = tweets_spam['text'].iloc[i]
    hwsl = [item for item in tw_msg.split() if item.isalpha()]
    val1 = ' '.join(hwsl)
    Sword_list.append(clean_text(val1))
print(Sword_list)

In [ ]:
print("===========================================================================")

In [ ]:
Hword_list = []

for ih in range(0, len(t_id_ham)):
    tw_msg_ham = tweets_ham['text'].iloc[ih]
    hwl = [item for item in tw_msg_ham.split() if item.isalpha()]
    val2 = ' '.join(hwl)
    Hword_list.append(clean_text(val2))
print(Hword_list)

In [ ]:
Hword_list

In [ ]:
spam_wordVector = pd.DataFrame(
    {'t_id': t_id,
     'words': Sword_list
    })

In [ ]:
spam_wordVector.head()

In [ ]:
ham_wordVector = pd.DataFrame(
    {'t_id': t_id_ham,
     'words': Hword_list
    })

In [ ]:
ham_wordVector.head()

# Vectorization

In [ ]:
from sklearn.feature_extraction.text  import CountVectorizer

In [ ]:
bow_transformerSpam = CountVectorizer(analyzer = text_process).fit(tweets_spam['text'])
bow_transformerHam = CountVectorizer(analyzer = text_process).fit(tweets_ham['text'])
bow_transformerTexts = CountVectorizer(analyzer = text_process).fit(tweets['text'])

In [ ]:
print(len(bow_transformerHam.vocabulary_))

In [ ]:
spam_words = list(bow_transformerSpam.vocabulary_.keys())
ham_words = list(bow_transformerHam.vocabulary_.keys())
tweets_words = list(bow_transformerTexts.vocabulary_.keys())

In [ ]:
texts_bow_spam = bow_transformerSpam.transform(tweets_spam['text'])
texts_bow_ham = bow_transformerHam.transform(tweets_ham['text'])
texts_bow = bow_transformerTexts.transform(tweets['text'])

In [ ]:
#get the text column 
docs_spam=tweets_spam['text'].tolist()
docs_ham=tweets_ham['text'].tolist()
docs=tweets['text'].tolist()
cv=CountVectorizer(max_df=0.85,stop_words=stopwords.words("english"),max_features=10000)

# Stemming

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
 
ps = PorterStemmer()

In [ ]:
print(spam_words)

In [ ]:
li_spamList = [item for item in spam_words if item.isalpha()]
li_hamList = [item for item in ham_words if item.isalpha()]

In [ ]:
print(li_hamList)

In [ ]:
print(li_spamList)

In [ ]:
spamWord = pd.DataFrame(li_spamList, columns=['words'])
hamWord = pd.DataFrame(li_hamList, columns=['words'])

In [ ]:
spamWord.to_csv('processed/spam_words.csv', index=False)
hamWord.to_csv('processed/ham_words.csv', index=False)

# Word Frequency

In [ ]:
Hword_list

In [ ]:
# li_spamList contains unique word in spam tweets
file = open("processed/spam_freq_word",'w')
file.write("spam_word,sword_count,hword_count")
file.write("\n")
word_set = set(li_spamList)
for worditem in word_set:
    scount = 0
    hcount = 0
    for sphraseitem in Sword_list:
        if worditem in sphraseitem:
            scount = scount + 1
    for hphraseitem in Hword_list:
        if worditem in hphraseitem:
            hcount = hcount + 1
    file.write(worditem+","+str(scount)+","+str(hcount))
    file.write("\n")
file.close()

In [ ]:
# li_hamList contains unique word in spam tweets
file1 = open("processed/ham_freq_word",'w')
file1.write("ham_word  hword_count  sword_count")
file1.write("\n")
hamword_set = set(li_hamList)
for hworditem in hamword_set:
    hcounter = 0
    scounter = 0
    for hphraseitem in Hword_list:
        if hworditem in hphraseitem:
            hcounter = hcounter + 1
    for sphraseitem in Sword_list:
        if hworditem in sphraseitem:
            scounter = scounter + 1
    file1.write(hworditem+"  "+str(hcounter)+"  "+str(scounter))
    file1.write("\n")
file1.close()

In [ ]:
spam_freq_word = pd.read_csv(r"processed/spam_freq_word")

In [ ]:
ham_freq_word = pd.read_csv(r"processed/ham_freq_word",sep="  ")

In [ ]:
spam_freq_word.dropna(axis=0, inplace=True)

In [ ]:
spam_freq_word['length'] = spam_freq_word['spam_word'].apply(len)

In [ ]:
ham_freq_word.dropna(axis=0, inplace=True)

In [ ]:
ham_freq_word['length'] = ham_freq_word['ham_word'].apply(len)

In [ ]:
spam_freq_word = spam_freq_word[spam_freq_word['length']>2]

In [ ]:
spam_freq_word.drop('length', axis=1, inplace=True)

In [ ]:
# word with count
spam_freq_word.to_csv("output/spam_freq_word.csv",index=False)

In [ ]:
ham_freq_word = ham_freq_word[ham_freq_word['length']>2]

In [ ]:
ham_freq_word.drop('length', axis=1, inplace=True)

In [ ]:
# word with count
ham_freq_word.to_csv("output/ham_freq_word.csv",index=False)

# Information Gain Calculation

In [ ]:
def calcualte_info_gain(ratio):
    #ratio = sprob / nsprob
    if ratio != 0.0:
        val = ratio * math.log10(ratio)
    else:
        val = 0.0
    return val

In [ ]:
#type(spam_freq_word)
type(ham_freq_word)

In [ ]:
spam_freq_word.head()

In [ ]:
SpamWordsCount = spam_freq_word.size
SpamWordsCount

In [ ]:
ham_freq_word.head()

In [ ]:
HamWordsCount = ham_freq_word.size
HamWordsCount

In [ ]:
#For Spam Data
#liSpam = list(spam_freq_word['spam_word'])
f_spam = open('output/sw_infoGain.csv', 'w')
f_spam.write("word,S_Prob,NS_Prob,Ratio,Info_Gain")
f_spam.write('\n')
for indx in range(0,len(spam_freq_word)):
    nparr = spam_freq_word.iloc[indx].values
    PtUs = round(nparr[1] / int(SpamWordsCount),6)
    PtUns = round(nparr[2] / int(HamWordsCount),6)
    ratio = round(PtUs / PtUns,6)
    ig = calcualte_info_gain(ratio)
    f_spam.write(nparr[0]+","+str(PtUs)+","+str(PtUns)+","+str(ratio)+","+str(ig))
    f_spam.write('\n')
f_spam.close()

In [ ]:
#For Ham Data
#liham = list(ham_freq_word['ham_word'])
f_ham = open('output/hw_infoGain.csv', 'w')
f_ham.write("word  S_Prob  NS_Prob  Ratio  Info_Gain")
f_ham.write('\n')
for index in range(0,len(ham_freq_word)):
    nparry = ham_freq_word.iloc[index].values
    PtUns = round(nparry[1] / int(HamWordsCount),6)
    PtUs = round(nparry[2] / int(SpamWordsCount),6)
    ratio = round(PtUs / PtUns,6)
    ig = calcualte_info_gain(ratio)
    f_ham.write(nparry[0]+"  "+str(PtUs)+"  "+str(PtUns)+"  "+str(ratio)+"  "+str(ig))
    f_ham.write('\n')
f_ham.close()

In [ ]:
df_sw_infoGain = pd.read_csv('output/sw_infoGain.csv')

In [ ]:
df_sw_infoGain.head(10)

In [ ]:
df_sw_infoGain.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df_sw_infoGain.dropna(inplace=True)

In [ ]:
df_sw_infoGain = df_sw_infoGain.sort_values(by='Info_Gain', ascending=False)

In [ ]:
df_sw_infoGain['label'] = 'spam'

In [ ]:
df_sw_infoGain.head(10)

In [ ]:
df_sw_infoGain.to_csv("output/sw_infoGain.csv")

In [ ]:
df_ham_infoGain = pd.read_csv('output/hw_infoGain.csv',sep="  ")

In [ ]:
df_ham_infoGain.head(10)

In [ ]:
df_ham_infoGain.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df_ham_infoGain.dropna(inplace=True)

In [ ]:
df_ham_infoGain = df_ham_infoGain.sort_values(by='Info_Gain', ascending=False)

In [ ]:
df_ham_infoGain['label'] = 'ham'

In [ ]:
df_ham_infoGain.head(10)

In [ ]:
df_ham_infoGain.to_csv("output/hw_infoGain.csv")

In [ ]:
df_spam_words = df_sw_infoGain.head(20)

In [ ]:
df_ham_words = df_ham_infoGain.head(20)

In [ ]:
type(spam_wordVector)

In [ ]:
type(ham_wordVector)

In [ ]:
spam_wordVector.iloc[0].values

In [ ]:
def check(string, sub_str): 
    if (string.lower().find(sub_str.lower()) == -1): 
        return 0  #No
    else: 
        return 1 #Yes

In [ ]:
def create_word_vector(nparray,df_word,label):
    # hold the word vector
    fv = []
    rid = nparray[0]
    msg = nparray[1]
    words = list(df_word['word'])
    #print(type(words))
    #fv.append(rid)
    for n in range(0,len(words)):
        fv.append(check(msg,words[n]))
    fv.append(label)
    return fv

In [ ]:
#For Spam
list_val=[]

for index in range(0,len(spam_wordVector)):
    rid = spam_wordVector.iloc[index].values[0]
    msg = spam_wordVector.iloc[index].values[1]
    list_val.append(create_word_vector(spam_wordVector.iloc[index],df_spam_words,1))

In [ ]:
list_val

In [ ]:
sp_cols = df_spam_words['word'].tolist()
sp_cols.append('label')
print(sp_cols)

In [ ]:
df_final_spam = pd.DataFrame(list_val,columns=sp_cols)

In [ ]:
df_final_spam.head(20)

In [ ]:
len(ham_wordVector)

In [ ]:
#For Ham
li_val=[]

for index in range(0,len(ham_wordVector)):
    #print(ham_wordVector.iloc[index].values)
    rid = ham_wordVector.iloc[index].values[0]
    msg = ham_wordVector.iloc[index].values[1]
    # label[spam] = 1 & label[ham] = 0
    li_val.append(create_word_vector(ham_wordVector.iloc[index],df_ham_words,0))

In [ ]:
li_val

In [ ]:
ham_cols = df_ham_words['word'].tolist()
ham_cols.append('label')
print(ham_cols)

In [ ]:
df_final_ham = pd.DataFrame(li_val,columns=ham_cols)

In [ ]:
df_final_ham.head(10)

In [ ]:
final_df = pd.merge(df_final_spam, df_final_ham, how='outer')

In [ ]:
final_df.fillna(0 ,inplace = True)

In [ ]:
final_df = final_df.reindex(np.random.permutation(final_df.index))

In [ ]:
final_df.head(10)

In [ ]:
final_df.to_csv('output/finalData.csv', index_label='Index')

## Data Splitting

In [ ]:
x = final_df.drop('label', axis=1)
y = final_df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
x_train.head(10)

In [ ]:
#print the shape of the data
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
# to save
x_train.to_csv('output/x_train.csv')
x_test.to_csv('output/x_test.csv')
y_train.to_csv('output/y_train.csv')
y_test.to_csv('output/y_test.csv')

In [ ]:
x_train

## Random Forest Classifier

In [ ]:
forest = RandomForestClassifier(n_estimators=50)

In [ ]:
forest.fit(x_train,y_train)

In [ ]:
y_pred = forest.predict(x_test)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
cm_plot_labels = [
    'ham',
    'spam'
]

In [ ]:
def plot_confusion_matrix(cm, classes,title='Confusion matrix'):
    
    plt.imshow(cm, interpolation='nearest', cmap='YlGnBu')
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
nb_cm = confusion_matrix(y_true=y_test,y_pred=y_pred)
plot_confusion_matrix(nb_cm, cm_plot_labels,title='Confusion matrix')

In [ ]:
import pickle as pkl

In [ ]:
pkl.dump(forest, open("model/rf_model.pkl","wb"))